In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
from cleverhans.tf2.attacks import fast_gradient_method as FGSM
from cleverhans.tf2.attacks import carlini_wagner_l2 as CW
from cleverhans.tf2.attacks import basic_iterative_method
from cleverhans.tf2.attacks import madry_et_al
from cleverhans.tf2.attacks import momentum_iterative_method
from cleverhans.tf2.attacks import spsa
from cleverhans.tf2.attacks import projected_gradient_descent

In [ ]:
df = pd.read_csv("Preprocessed_data.csv")


In [ ]:
df_tensor_input=tf.convert_to_tensor(df.drop(["frame.number","normality"], axis=1))
df_tensor_output=tf.convert_to_tensor(df["normality"])

In [ ]:
df_tensor_input.shape

In [ ]:
input_dim=df_tensor_input.shape[1]

inputs = tf.keras.Input(shape=(input_dim,))
x = tf.keras.layers.Dense(units=8, activation="sigmoid")(inputs)
outputs = tf.keras.layers.Dense(units=5, activation="sigmoid")(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(optimizer='adam',
              loss= 'mean_squared_error',
              metrics=['accuracy'])

model.summary()

In [ ]:
model.fit(df_tensor_input, df_tensor_output, epochs=10)

In [ ]:
import tensorflow as tf

normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(df_tensor_input)

(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.imdb.load_data()

model = tf.keras.models.Sequential([
    normalizer,
    tf.keras.layers.Dense(32, activation=tf.nn.relu),
    tf.keras.layers.Dense(16, activation=tf.nn.relu),
    tf.keras.layers.Dense(10),
    tf.keras.layers.Activation(tf.nn.softmax) # We seperate the activation layer to be able to access the logits of the previous layer later
])

model.compile(optimizer='adam',
              loss= 'sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(df_tensor_input, df_tensor_output)

In [ ]:
adv = FGSM.fast_gradient_method(model, df_tensor_input, 0.05, 1)

In [ ]:
import cleverhans
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images / 255.0
test_images = test_images / 255.0

num_classes = 10

model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(32, activation=tf.nn.relu),
    tf.keras.layers.Dense(16, activation=tf.nn.relu),
    tf.keras.layers.Dense(10),
    tf.keras.layers.Activation(tf.nn.softmax) # We seperate the activation layer to be able to access the logits of the previous layer later
])

model.compile(optimizer='adam',
              loss= 'sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=10, validation_split=0.2)
test_loss, test_acc = model.evaluate(test_images, test_labels)

print('Test accuracy:', test_acc)

In [ ]:
from cleverhans.tf2.attacks import fast_gradient_method as FGSM

#The attack requires the model to ouput the logits
logits_model = tf.keras.Model(model.input,model.layers[-1].output)

In [ ]:
random_index = np.random.randint(test_images.shape[0])

original_image = test_images[random_index]
original_image = tf.convert_to_tensor(original_image.reshape((1,28,28))) #The .reshape just gives it the proper form to input into the model, a batch of 1 a.k.a a tensor

original_label = test_labels[random_index]
original_label = np.reshape(original_label, (1,)).astype('int64') # Give label proper shape and type for cleverhans

#Show the image
plt.figure()
plt.grid(False)

plt.imshow(np.reshape(original_image, (28,28)))
plt.title("Label: {}".format(original_label[0]))

plt.show()

In [ ]:
epsilon = 0.1

adv_example_untargeted_label = FGSM.fast_gradient_method(
    model_fn=model, 
    x=original_image, 
    eps=epsilon, 
    norm=np.inf, 
    targeted=False)

adv_example_untargeted_label_pred = model.predict(adv_example_untargeted_label)

#Show the image
plt.figure()
plt.grid(False)

plt.imshow(np.reshape(adv_example_untargeted_label, (28,28)))
plt.title("Model Prediction: {}".format(np.argmax(adv_example_untargeted_label_pred)))
plt.xlabel("Original Label: {}".format(original_label[0]))

plt.show()